## Import modules & set database connection

In [1]:
%load_ext sql
import pandas as pd
import sqlite3 as sql

%sql sqlite:///CourseData.db
conn = sql.connect('CourseData.db')

## Domain Integrity

In [2]:
%%sql
SELECT * FROM COURSE_OFFERINGS;
PRAGMA table_info(COURSE_OFFERINGS);

 * sqlite:///CourseData.db
Done.
Done.


cid,name,type,notnull,dflt_value,pk
0,CourseOfferingID,INTEGER,0,None,1
1,CourseID,INTEGER,0,None,0
2,CatalogID,TEXT,1,None,0
3,Term,TEXT,1,None,0
4,CRN,INTEGER,1,None,0
5,Section,TEXT,1,None,0
6,Credits,REAL,0,None,0
7,Title,TEXT,1,None,0
8,Timecodes,TEXT,0,None,0
9,PrimaryInstructorID,INTEGER,0,None,0


In [5]:
%%sql
SELECT * FROM CATALOG_COURSES;
PRAGMA table_info(CATALOG_COURSES);

 * sqlite:///CourseData.db
Done.
Done.


cid,name,type,notnull,dflt_value,pk
0,CourseID,INTEGER,0,None,1
1,CatalogYear,TEXT,1,None,0
2,CatalogID,TEXT,1,None,0
3,ProgramID,INTEGER,0,None,0
4,CourseTitle,TEXT,1,None,0
5,Credits,TEXT,1,None,0
6,Prereqs,TEXT,0,None,0
7,Coreqs,TEXT,0,None,0
8,Fees,TEXT,0,None,0
9,Attributes,TEXT,0,None,0


In [6]:
%%sql
SELECT * FROM COURSE_MEETINGS;
PRAGMA table_info(COURSE_MEETINGS);

 * sqlite:///CourseData.db
Done.
Done.


cid,name,type,notnull,dflt_value,pk
0,CourseMeetingID,INTEGER,0,None,1
1,CourseOfferingID,INTEGER,1,None,0
2,LocationID,INTEGER,1,None,0
3,StartDateTime,TEXT,1,None,0
4,EndDateTime,TEXT,1,None,0


## Entity Integrity

- There should be 4440 Catalog Entries, 15937 Course Offerings, and 284847 Course Meetings. Added the rest of the tables as a "sanity" double check since the 3 mentioned create the "core" of our database.
- With our Entity integrity check we are expecting to match the number of rows within our tables to the number of rows that we inserted into our CourseData.db.

In [7]:
%%sql

SELECT 
    (SELECT Count(*) FROM CATALOG_COURSES) as CatalogCourses,
    (SELECT Count(*) FROM COURSE_OFFERINGS) as CourseOfferings,
    (SELECT Count(*) FROM COURSE_MEETINGS) as CourseMeetings,
    (SELECT Count(*) FROM PROGRAMS) as Programs,
    (SELECT Count(*) FROM TERM_TO_CATALOG_YEAR) as TermToCatYear,
    (SELECT Count(*) FROM COURSE_INSTRUCTORS) as Course_Instructors,
    (SELECT Count(*) FROM COURSE_LOCATIONS) as Course_Locations   
    ;

 * sqlite:///CourseData.db
Done.


CatalogCourses,CourseOfferings,CourseMeetings,Programs,TermToCatYear,Course_Instructors,Course_Locations
4440,15937,284847,83,7,1095,207


## Relational Integrity

- Here we are joining all of the tables within our ERD and utilizing fields from different tables to show a high level example of what is possible to derive from our database

In [9]:
%%sql

SELECT InstructorID,Name as ProfessorName,Term,Section,Title as CourseTitle,substr(StartDateTime,1,10) as CourseDate,substr(StartDateTime,-8) as CourseStartTime,substr(EndDateTime,-8) as CourseEndTime,LocationCode as Location,substr(Term,-4) as Year 
FROM COURSE_OFFERINGS
    LEFT JOIN COURSE_INSTRUCTORS ON (COURSE_OFFERINGS.PrimaryInstructorID = COURSE_INSTRUCTORS.InstructorID)
    LEFT JOIN TERM_TO_CATALOG_YEAR USING(Term)
    LEFT JOIN CATALOG_COURSES USING(CourseID)
    LEFT JOIN PROGRAMS USING(ProgramID)
    LEFT JOIN COURSE_MEETINGS USING(CourseOfferingID)
    LEFT JOIN COURSE_LOCATIONS USING(LocationID)
WHERE Name LIKE '%Huntley'
ORDER BY Term DESC
LIMIT 10;



 * sqlite:///CourseData.db
Done.


InstructorID,ProfessorName,Term,Section,CourseTitle,CourseDate,CourseStartTime,CourseEndTime,Location,Year
272,Christopher L. Huntley,Spring2019,02,Operations Management,2019-01-24,11:00:00,12:15:00,DSB 110B,2019
272,Christopher L. Huntley,Spring2019,02,Operations Management,2019-01-28,11:00:00,12:15:00,DSB 110B,2019
272,Christopher L. Huntley,Spring2019,02,Operations Management,2019-01-31,11:00:00,12:15:00,DSB 110B,2019
272,Christopher L. Huntley,Spring2019,02,Operations Management,2019-02-04,11:00:00,12:15:00,DSB 110B,2019
272,Christopher L. Huntley,Spring2019,02,Operations Management,2019-02-07,11:00:00,12:15:00,DSB 110B,2019
272,Christopher L. Huntley,Spring2019,02,Operations Management,2019-02-11,11:00:00,12:15:00,DSB 110B,2019
272,Christopher L. Huntley,Spring2019,02,Operations Management,2019-02-14,11:00:00,12:15:00,DSB 110B,2019
272,Christopher L. Huntley,Spring2019,02,Operations Management,2019-02-19,11:00:00,12:15:00,DSB 110B,2019
272,Christopher L. Huntley,Spring2019,02,Operations Management,2019-02-21,11:00:00,12:15:00,DSB 110B,2019
272,Christopher L. Huntley,Spring2019,02,Operations Management,2019-02-25,11:00:00,12:15:00,DSB 110B,2019
